Training notebook [fgvc8_fastai_training1](https://www.kaggle.com/teykaihong/fgvc8-fastai)

In [ ]:
from fastai.vision.all import *

class CFG:
    models = ['../input/trained-models1/alexnet.pkl', '../input/trained-models1/resnet50.pkl']
    test_images_path = Path('../input/plant-pathology-2021-fgvc8/test_images')
    threshs_path = Path('../input/trained-models1/thresholds.h5')
    classes = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
    tta_n = 3
    
def get_x(x):
    return CFG.test_images_path/x['image']

def get_y(y):
    return y['labels'].split(' ')

def get_threshs(path):
    import h5py
    threshs = []
    
    with h5py.File(path, 'r') as h:
        threshs.append(np.array(h['thresholds']))
        
    return (threshs[0])

def to_labels(preds, threshs):
    labels = []
    for pred in preds:
        names = []
        for idx, ret in enumerate((pred.tolist()>threshs).astype(int)):
            if ret:
                names.append(CFG.classes[idx])
        labels.append(' '.join(names))
    
    for n_pred, label in enumerate(labels):
        if label == '':
            min_id = 0
            global_min = 2
            for cls_id, val in enumerate(abs(preds[n_pred].tolist() - threshs)):
                local_min = val
                if local_min < global_min:
                    min_id = cls_id
                    global_min = local_min
                    
            labels[n_pred] = CFG.classes[min_id]
            
    return labels
    
def csv_formation(img_path):
    
    image_names = os.listdir(img_path)
    df = pd.DataFrame({'image': image_names})
    
    return df

def submit(df, labels):
    df['labels'] = labels
    df.to_csv('./submission.csv', index=False)
    display(df)

In [ ]:
predictions = 0
test_df = csv_formation(CFG.test_images_path)
threshs = get_threshs(CFG.threshs_path)

for m in CFG.models:
    learner = load_learner(m, cpu=False).to_fp32()
    test_dl = learner.dls.test_dl(test_df)
    preds, _ = learner.tta(dl=test_dl, n=CFG.tta_n)
    predictions += preds

predictions /= len(CFG.models)
labels = to_labels(predictions, threshs)
submit(test_df, labels)